In [1]:
import os
import re
import pandas as pd

In [2]:
results_folder = [os.pardir,"Benchmark results","halloc_opts","halloc_results_crash"]
# results_folder = [os.pardir,"Benchmark results"]
print(os.path.join(*results_folder))


../Benchmark results/halloc_opts/halloc_results_crash


In [3]:
parsable_keys = {
    "HALLOC_FRACTION": "hf-",
    "BUSY_FRACTION": "bu-",
    "ROOMY_FRACTION": "ro-",
    "SPARSE_FRACTION": "sp-",
    "MAX_BLOCK_SZ": "mb-",
    "MAX_NSIZES": "ns-",
    "MAX_NCHUNK_IDS": "nc-",
    "BLOCK_STEP": "bst-",
    "NUNITS": "nu-",
    "MIN_BLOCK_SZ": "mibs-",
    "MAX_BLOCK_SZ": "mabs-",
}

In [4]:
default_params = {
    "HALLOC_FRACTION": 0.25,
    "BUSY_FRACTION": 0.835,
    "ROOMY_FRACTION": 0.6,
    "SPARSE_FRACTION": 0.012,
    "MAX_BLOCK_SZ": 3072,
    "MAX_NSIZES": 16,
    "MAX_NCHUNK_IDS": 8,
    "BLOCK_STEP": 16,
    "NUNITS": 8,
    "MIN_BLOCK_SZ": 16,
    "MAX_BLOCK_SZ": 3072,
}

In [5]:
expr = r'total\s+images\/sec:\s+(\d+\.*\d*)'
r = re.compile(expr)

In [6]:
cols = ["ALLOCATOR", "NETWORK", "IMAGES_SEC", "PROFILING"] + list(parsable_keys.keys())
df = pd.DataFrame(columns=cols)
print(df)

Empty DataFrame
Columns: [ALLOCATOR, NETWORK, IMAGES_SEC, PROFILING, HALLOC_FRACTION, BUSY_FRACTION, ROOMY_FRACTION, SPARSE_FRACTION, MAX_BLOCK_SZ, MAX_NSIZES, MAX_NCHUNK_IDS, BLOCK_STEP, NUNITS, MIN_BLOCK_SZ]
Index: []


In [7]:
for file in os.listdir(os.path.join(*results_folder)):
#     print(file)
	file_name = os.path.splitext(file)
	if file_name[-1] == ".txt":
		params = default_params.copy()
# 		print(file_name[0].split('_'))
		
		file_split = file_name[0].split('_')
		params["ALLOCATOR"] = file_split.pop(0)
		params["NETWORK"] = file_split.pop(0)
		params["PROFILING"] = "prof" in file_split
		for part in file_split:
			for key, value in parsable_keys.items():
				if value in part:
# 					print(key, part.split('-')[-1])
					params[key]=float(part.split('-')[-1])
# 		print(params)
		
		matches = [re.findall(expr,line) for line in open(os.path.join(*results_folder, file))]
		matches = [float(x[0]) for x in matches if len(x)]
# 		print(matches)
		
		if len(matches):
			for match in matches:
				params["IMAGES_SEC"] = match
				df = df.append(params, ignore_index=True)
		else:
			params["IMAGES_SEC"] = float('NaN')
			df = df.append(params, ignore_index=True)
df

,ALLOCATOR,NETWORK,IMAGES_SEC,PROFILING,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION,MAX_BLOCK_SZ,MAX_NSIZES,MAX_NCHUNK_IDS,BLOCK_STEP,NUNITS,MIN_BLOCK_SZ
0,halloc,alex,36.32,False,0.25,0.835,0.60,0.200,3072,16,8,16,8,16
1,halloc,resnet,NaN,False,0.25,0.700,0.60,0.012,3072,16,8,16,8,16
2,halloc,resnet,NaN,False,0.25,0.835,0.80,0.012,3072,16,8,16,8,16
3,halloc,resnet,NaN,False,0.25,0.900,0.60,0.012,3072,16,8,16,8,16
4,halloc,resnet,NaN,False,0.70,0.835,0.60,0.012,3072,16,8,16,8,16
5,halloc,resnet,NaN,False,0.25,0.835,0.75,0.012,3072,16,8,16,8,16
6,halloc,alex,NaN,False,0.25,0.835,0.60,0.012,3072,2,8,16,8,16
7,halloc,resnet,NaN,False,0.25,0.835,0.60,0.012,3072,16,2,16,8,16
8,halloc,resnet,NaN,False,0.25,0.750,0.60,0.012,3072,16,8,16,8,16
9,halloc,alex,36.30,False,0.25,0.850,0.60,0.012,3072,16,8,16,8,16


In [8]:
df.to_csv(os.path.join(*results_folder,'results.csv'))

In [9]:
df.describe()

,IMAGES_SEC,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION
count,58.000000,148.000000,148.000000,148.000000,148.000000
mean,35.758793,0.287162,0.832635,0.604730,0.025243
std,4.187394,0.151424,0.020441,0.034175,0.039738
min,4.420000,0.000000,0.700000,0.500000,0.012000
25%,36.290000,0.250000,0.835000,0.600000,0.012000
50%,36.310000,0.250000,0.835000,0.600000,0.012000
75%,36.330000,0.250000,0.835000,0.600000,0.012000
max,36.380000,1.000000,0.900000,0.800000,0.200000


In [10]:
df[((df.NETWORK == "alex") & (df.HALLOC_FRACTION < 0.25))]

,ALLOCATOR,NETWORK,IMAGES_SEC,PROFILING,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION,MAX_BLOCK_SZ,MAX_NSIZES,MAX_NCHUNK_IDS,BLOCK_STEP,NUNITS,MIN_BLOCK_SZ
51,halloc,alex,36.31,False,0.2,0.835,0.6,0.012,3072,16,8,16,8,16
74,halloc,alex,36.27,False,0.1,0.835,0.6,0.012,3072,16,8,16,8,16
78,halloc,alex,36.33,False,0.0,0.835,0.6,0.012,3072,16,8,16,8,16


In [11]:
# df[df.NETWORK == "resnet"].describe()

In [12]:
from sklearn import linear_model

df2 = df[df.NETWORK == "alex"]

reg = linear_model.LinearRegression()
reg.fit(df2[["PROFILING", "HALLOC_FRACTION", "BUSY_FRACTION", "ROOMY_FRACTION", "SPARSE_FRACTION", "MAX_BLOCK_SZ", "MAX_NSIZES", "MAX_NCHUNK_IDS", "BLOCK_STEP", "NUNITS", "MIN_BLOCK_SZ" , "MAX_BLOCK_SZ"]], df2["IMAGES_SEC"])

print(reg.coef_)
# print(reg.intercept_)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').